Adapted from: https://github.com/alphacep/vosk-api/blob/master/python/example/colab/vosk-adaptation.ipynb

Official Documentation: https://alphacephei.com/vosk/lm

Vosk models are compiled into FST graphs.

The following code modifies them to adapt to a domain (boost probability of domain related words)

# Install libraries

In [1]:
!pip install phonetisaurus

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 4.2 MB/s eta 0:00:00


In [2]:
!apt install flac

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  flac
0 upgraded, 1 newly installed, 0 to remove and 29 not upgraded.
Need to get 130 kB of archives.
After this operation, 359 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 flac amd64 1.3.3-2ubuntu0.2 [130 kB]
Fetched 130 kB in 0s (284 kB/s)
Selecting previously unselected package flac.
(Reading database ... 126209 files and directories currently installed.)
Preparing to unpack .../flac_1.3.3-2ubuntu0.2_amd64.deb ...
Unpacking flac (1.3.3-2ubuntu0.2) ...
Setting up flac (1.3.3-2ubuntu0.2) ...
Processing triggers for man-db (2.10.2-1) ...


In [40]:
!apt install nvidia-cuda-toolkit --yes

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libaccinj64-11.5 libatk-wrapper-java libatk-wrapper-java-jni
  libbabeltrace1 libcub-dev libcublas11 libcublaslt11 libcudart11.0 libcufft10 libcufftw10
  libcuinj64-11.5 libcupti-dev libcupti-doc libcupti11.5 libcurand10 libcusolver11 libcusolvermg11
  libcusparse11 libdebuginfod-common libdebuginfod1 libegl-dev libfontenc1 libgail-common libgail18
  libgl-dev libgl1-mesa-dev libgles-dev libgles1 libglvnd-core-dev libglvnd-dev libglx-dev
  libgtk2.0-0 libgtk2.0-bin libgtk2.0-common libipt2 libnppc11 libnppial11 libnppicc11 libnppidei11
  libnppif11 libnppig11 libnppim11 libnppist11 libnppisu11 libnppitc11 libnpps11 libnvblas11
  libnvidia-compute-495 libnvidia-compute-510 libnvidia-compute-535 libnvidia-ml-dev libnvjpeg11
  libnvrtc-builtins11.5 libnvrtc11.2 libnvtoolsext1 libnvvm4 libopengl-dev libr

# Download update package

In [3]:
%cd /content/
# %cd /content
!wget -c https://alphacephei.com/vosk-colab/kaldi.tar.gz
!tar xzf kaldi.tar.gz

/content
--2025-03-27 05:29:17--  https://alphacephei.com/vosk-colab/kaldi.tar.gz
Resolving alphacephei.com (alphacephei.com)... 188.40.21.16, 2a01:4f8:13a:279f::2
Connecting to alphacephei.com (alphacephei.com)|188.40.21.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1041599901 (993M) [application/octet-stream]
Saving to: ‘kaldi.tar.gz’

kaldi.tar.gz        100%[===================>] 993.35M   102MB/s    in 13s     

2025-03-27 05:29:31 (74.4 MB/s) - ‘kaldi.tar.gz’ saved [1041599901/1041599901]



In [6]:
%cd /content/kaldi/egs/ac
!wget -c https://alphacephei.com/vosk-colab/vosk-model-small-en-us-0.15-compile-colab.tar.gz
!rm -rf vosk-model-small-en-us-0.15-compile-colab
!tar xf vosk-model-small-en-us-0.15-compile-colab.tar.gz

/content/kaldi/egs/ac
--2025-03-27 05:32:23--  https://alphacephei.com/vosk-colab/vosk-model-small-en-us-0.15-compile-colab.tar.gz
Resolving alphacephei.com (alphacephei.com)... 188.40.21.16, 2a01:4f8:13a:279f::2
Connecting to alphacephei.com (alphacephei.com)|188.40.21.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59618100 (57M) [application/octet-stream]
Saving to: ‘vosk-model-small-en-us-0.15-compile-colab.tar.gz’

vosk-model-small-en 100%[===================>]  56.86M  90.5MB/s    in 0.6s    

2025-03-27 05:32:23 (90.5 MB/s) - ‘vosk-model-small-en-us-0.15-compile-colab.tar.gz’ saved [59618100/59618100]



# Configure domain words

In [7]:
domain_words = """biscuit, battery, milk, bread, cheese, butter, eggs, yogurt, cereal, juice, soda, coffee, tea, sugar, flour, rice, pasta, noodles, oil, salt, pepper, ketchup, mayonnaise, mustard, jam, honey
chocolate, candy, chips, crackers, nuts, granola, water, beer, wine, whiskey, vodka, chicken, beef, pork, fish, shrimp, sausage, bacon, ham, tofu, lettuce, tomato, cucumber, onion, garlic, potato, carrot, apple, banana, orange, grapes
watermelon, pineapple, mango, avocado, detergent, soap, shampoo, toothpaste, toothbrush, tissue, paper towel, toilet paper, razor, deodorant, lotion, sunscreen, diaper, baby wipes
dog food, cat food, laundry pods, dish soap, sponges, batteries, light bulb, matches, foil, plastic wrap, trash bags, mop, broom, candles, air freshener, vitamins, medicine, band-aid
energy drink, protein bar"""
domain_words = domain_words.replace(", ","\n")
print(domain_words)

# with open("/content/kaldi/egs/ac/vosk-model-en-us-0.22-compile/db/extra.txt","w") as f:
with open("/content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab/db/extra.txt","w") as f:
    f.write(domain_words)

biscuit
battery
milk
bread
cheese
butter
eggs
yogurt
cereal
juice
soda
coffee
tea
sugar
flour
rice
pasta
noodles
oil
salt
pepper
ketchup
mayonnaise
mustard
jam
honey
chocolate
candy
chips
crackers
nuts
granola
water
beer
wine
whiskey
vodka
chicken
beef
pork
fish
shrimp
sausage
bacon
ham
tofu
lettuce
tomato
cucumber
onion
garlic
potato
carrot
apple
banana
orange
grapes
watermelon
pineapple
mango
avocado
detergent
soap
shampoo
toothpaste
toothbrush
tissue
paper towel
toilet paper
razor
deodorant
lotion
sunscreen
diaper
baby wipes
dog food
cat food
laundry pods
dish soap
sponges
batteries
light bulb
matches
foil
plastic wrap
trash bags
mop
broom
candles
air freshener
vitamins
medicine
band-aid
energy drink
protein bar


Confirm content of extra.txt

In [8]:
!cat /content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab/db/extra.txt

biscuit
battery
milk
bread
cheese
butter
eggs
yogurt
cereal
juice
soda
coffee
tea
sugar
flour
rice
pasta
noodles
oil
salt
pepper
ketchup
mayonnaise
mustard
jam
honey
chocolate
candy
chips
crackers
nuts
granola
water
beer
wine
whiskey
vodka
chicken
beef
pork
fish
shrimp
sausage
bacon
ham
tofu
lettuce
tomato
cucumber
onion
garlic
potato
carrot
apple
banana
orange
grapes
watermelon
pineapple
mango
avocado
detergent
soap
shampoo
toothpaste
toothbrush
tissue
paper towel
toilet paper
razor
deodorant
lotion
sunscreen
diaper
baby wipes
dog food
cat food
laundry pods
dish soap
sponges
batteries
light bulb
matches
foil
plastic wrap
trash bags
mop
broom
candles
air freshener
vitamins
medicine
band-aid
energy drink
protein bar

Empty content from extra.dic example file

In [9]:
with open("/content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab/db/extra.dic","w") as f:
    f.write("")

In [11]:
!cat /content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab/compile-graph.sh

#!/bin/bash

set -x

. path.sh

pip3 install phonetisaurus

rm -rf data
rm -rf exp/tdnn/lgraph
rm -rf exp/tdnn/lgraph_orig

mkdir -p data/dict
cp db/phone/* data/dict
./dict.py > data/dict/lexicon.txt

python3 ./get_vocab.py > data/mix.vocab
ngramsymbols data/mix.vocab data/mix.syms
farcompilestrings --fst_type=compact --symbols=data/mix.syms --keep_symbols --unknown_symbol="[unk]" db/extra.txt data/extra.far
ngramcount --order=3 data/extra.far - |
    ngramprint --integers | grep -v "<unk>" | ngramread |
    ngramshrink --method=count_prune --count_pattern="3+:3" |
    ngrammake --method=witten_bell - data/extra.mod
gunzip -c db/en-50k-0.4-android.lm.gz | ngramread --renormalize_arpa --ARPA --symbols=data/mix.syms - data/en-us.mod
ngrammerge --method="bayes_model_merge" --normalize --alpha=0.95 --beta=0.05 data/en-us.mod data/extra.mod data/en-us-mix.mod
ngramprint --ARPA data/en-us-mix.mod | gzip -c > data/en-us-mix.lm.gz

# Prune for the first stage if needed
# ngramshrink --method=

# Run mkgraph

Uncomment lines for mkgraph for small model

In [27]:
with open("/content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab/compile-graph.sh", "w") as f:
    f.write("""#!/bin/bash

set -x

. path.sh

pip3 install phonetisaurus

rm -rf data
rm -rf exp/tdnn/lgraph
rm -rf exp/tdnn/lgraph_orig

mkdir -p data/dict
cp db/phone/* data/dict
./dict.py > data/dict/lexicon.txt

python3 ./get_vocab.py > data/mix.vocab
ngramsymbols data/mix.vocab data/mix.syms
farcompilestrings --fst_type=compact --symbols=data/mix.syms --keep_symbols --unknown_symbol="[unk]" db/extra.txt data/extra.far
ngramcount --order=3 data/extra.far - |
    ngramprint --integers | grep -v "<unk>" | ngramread |
    ngramshrink --method=count_prune --count_pattern="3+:3" |
    ngrammake --method=witten_bell - data/extra.mod
gunzip -c db/en-50k-0.4-android.lm.gz | ngramread --renormalize_arpa --ARPA --symbols=data/mix.syms - data/en-us.mod
ngrammerge --method="bayes_model_merge" --normalize --alpha=0.95 --beta=0.05 data/en-us.mod data/extra.mod data/en-us-mix.mod
ngramprint --ARPA data/en-us-mix.mod | gzip -c > data/en-us-mix.lm.gz

# Prune for the first stage if needed
# ngramshrink --method=relative_entropy --theta=2e-8 data/en-us-mix.mod data/en-us-mix-prune.mod
# ngramprint --ARPA data/en-us-mix-prune.mod | gzip -c > data/en-us-mix-small.lm.gz

utils/prepare_lang.sh data/dict "[unk]" data/lang_local data/lang
utils/format_lm.sh data/lang db/en-50k-0.4-android.lm.gz data/dict/lexicon.txt data/lang_test
utils/format_lm.sh data/lang data/en-us-mix.lm.gz data/dict/lexicon.txt data/lang_test_adapt

utils/mkgraph.sh --self-loop-scale 1.0 data/lang_test exp/tdnn exp/tdnn/graph
utils/mkgraph.sh --self-loop-scale 1.0 data/lang_test_adapt exp/tdnn exp/tdnn/graph_adapt

echo "======================"
# Lookahead part goes OOM
utils/mkgraph_lookahead.sh \
         --self-loop-scale 1.0 data/lang \
         exp/tdnn data/en-us-mix.lm.gz exp/tdnn/lgraph
utils/mkgraph_lookahead.sh \
         --self-loop-scale 1.0 data/lang \
         exp/tdnn db/en-50k-0.4-android.lm.gz exp/tdnn/lgraph_orig
""")

In [41]:
%cd /content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab
!bash compile-graph.sh

/content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab
+ . path.sh
+++ pwd
++ export KALDI_ROOT=/content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab/../../..
++ KALDI_ROOT=/content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab/../../..
++ export PATH=/content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab/utils:/content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab/../../../src/bin:/content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab/../../../tools/openfst/bin:/content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab/../../../src/fstbin:/content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab/../../../src/gmmbin:/content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab/../../../src/featbin:/content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab/../../../src/lm:/content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab/../../../src/sgmmbin:/content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-co

In [33]:
!utils/mkgraph_lookahead.sh --self-loop-scale 1.0 data/lang exp/tdnn data/en-us-mix.lm.gz exp/tdnn/lgraph

utils/mkgraph_lookahead.sh : compiling grammar data/en-us-mix.lm.gz
tree-info exp/tdnn/tree 
tree-info exp/tdnn/tree 
fstdeterminizestar data/lang/L_disambig.fst 
fstcomposecontext --context-size=2 --central-position=1 --read-disambig-syms=data/lang/phones/disambig.int --write-disambig-syms=exp/tdnn/lgraph/disambig_ilabels_2_1.int exp/tdnn/lgraph/ilabels_2_1.11787 exp/tdnn/lgraph/L_disambig_det.fst 
make-h-transducer --disambig-syms-out=exp/tdnn/lgraph/disambig_tid.int --transition-scale=1.0 exp/tdnn/lgraph/ilabels_2_1 exp/tdnn/tree exp/tdnn/final.mdl 
add-self-loops --disambig-syms=exp/tdnn/lgraph/disambig_tid.int --self-loop-scale=1.0 --reorder=true exp/tdnn/final.mdl 
fstdeterminizestar 


In [34]:
!utils/mkgraph_lookahead.sh --self-loop-scale 1.0 data/lang exp/tdnn db/en-50k-0.4-android.lm.gz exp/tdnn/lgraph_orig

utils/mkgraph_lookahead.sh : compiling grammar db/en-50k-0.4-android.lm.gz
tree-info exp/tdnn/tree 
tree-info exp/tdnn/tree 
fstdeterminizestar data/lang/L_disambig.fst 
fstcomposecontext --context-size=2 --central-position=1 --read-disambig-syms=data/lang/phones/disambig.int --write-disambig-syms=exp/tdnn/lgraph_orig/disambig_ilabels_2_1.int exp/tdnn/lgraph_orig/ilabels_2_1.11847 exp/tdnn/lgraph_orig/L_disambig_det.fst 
make-h-transducer --disambig-syms-out=exp/tdnn/lgraph_orig/disambig_tid.int --transition-scale=1.0 exp/tdnn/lgraph_orig/ilabels_2_1 exp/tdnn/tree exp/tdnn/final.mdl 
add-self-loops --disambig-syms=exp/tdnn/lgraph_orig/disambig_tid.int --self-loop-scale=1.0 --reorder=true exp/tdnn/final.mdl 
fstdeterminizestar 


# Decode results

Uncomment lines for decoding small model

In [42]:
with open("/content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab/decode.sh", "w") as f:
    f.write("""#!/bin/bash

. path.sh

steps/make_mfcc.sh --nj 10 data_test/test_small exp/make_mfcc/test mfcc
steps/compute_cmvn_stats.sh data_test/test_small exp/make_mfcc/test mfcc
utils/fix_data_dir.sh data_test/test_small

steps/online/nnet2/extract_ivectors_online.sh --nj 4 \
          data_test/test_small exp/extractor \
          exp/ivectors_test

steps/nnet3/decode.sh --nj 4 \
          --acwt 1.0 --post-decode-acwt 10.0 \
          --online-ivector-dir exp/ivectors_test \
          exp/tdnn/graph_adapt data_test/test_small exp/tdnn/decode_test_adapt

steps/nnet3/decode.sh --nj 4 \
          --acwt 1.0 --post-decode-acwt 10.0 \
          --online-ivector-dir exp/ivectors_test \
          exp/tdnn/graph data_test/test_small exp/tdnn/decode_test

steps/nnet3/decode_lookahead.sh --nj 4 \
         --acwt 1.0 --post-decode-acwt 10.0 \
         --online-ivector-dir exp/ivectors_test \
         exp/tdnn/lgraph data_test/test_small exp/tdnn/decode_test_adapt
steps/nnet3/decode_lookahead.sh --nj 4 \
         --acwt 1.0 --post-decode-acwt 10.0 \
         --online-ivector-dir exp/ivectors_test \
         exp/tdnn/lgraph_orig data_test/test_small exp/tdnn/decode_test""")

In [35]:
!cat "/content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab/decode.sh"

#!/bin/bash

. path.sh

steps/make_mfcc.sh --nj 10 data_test/test_small exp/make_mfcc/test mfcc
steps/compute_cmvn_stats.sh data_test/test_small exp/make_mfcc/test mfcc
utils/fix_data_dir.sh data_test/test_small

steps/online/nnet2/extract_ivectors_online.sh --nj 4           data_test/test_small exp/extractor           exp/ivectors_test

steps/nnet3/decode.sh --nj 4           --acwt 1.0 --post-decode-acwt 10.0           --online-ivector-dir exp/ivectors_test           exp/tdnn/graph_adapt data_test/test_small exp/tdnn/decode_test_adapt

steps/nnet3/decode.sh --nj 4           --acwt 1.0 --post-decode-acwt 10.0           --online-ivector-dir exp/ivectors_test           exp/tdnn/graph data_test/test_small exp/tdnn/decode_test

steps/nnet3/decode_lookahead.sh --nj 4          --acwt 1.0 --post-decode-acwt 10.0          --online-ivector-dir exp/ivectors_test          exp/tdnn/lgraph data_test/test_small exp/tdnn/decode_test_adapt
steps/nnet3/decode_lookahead.sh --nj 4          --acwt 1.0 --p

In [43]:
%cd /content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab
# !cat decode.sh
# print("*" * 50)
!bash decode.sh

/content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab
steps/make_mfcc.sh --nj 10 data_test/test_small exp/make_mfcc/test mfcc
steps/make_mfcc.sh: moving data_test/test_small/feats.scp to data_test/test_small/.backup
utils/validate_data_dir.sh: Successfully validated data-directory data_test/test_small
steps/make_mfcc.sh: [info]: no segments file exists: assuming wav.scp indexed by utterance.
steps/make_mfcc.sh: Succeeded creating MFCC features for test_small
steps/compute_cmvn_stats.sh data_test/test_small exp/make_mfcc/test mfcc
Succeeded creating CMVN stats for test_small
fix_data_dir.sh: kept all 50 utterances.
fix_data_dir.sh: old files are kept in data_test/test_small/.backup
steps/online/nnet2/extract_ivectors_online.sh --nj 4 data_test/test_small exp/extractor exp/ivectors_test
steps/online/nnet2/extract_ivectors_online.sh: extracting iVectors
steps/online/nnet2/extract_ivectors_online.sh: combining iVectors across jobs
steps/online/nnet2/extract_ivectors_online.sh: do

# Display results

In [44]:
!bash RESULTS

%WER 11.77 [ 107 / 909, 13 ins, 7 del, 87 sub ] exp/tdnn/decode_test/wer_7_1.0
%WER 11.88 [ 108 / 909, 13 ins, 7 del, 88 sub ] exp/tdnn/decode_test_adapt/wer_7_1.0


In [45]:
!ls /content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab/

compile-graph.sh  data	     db		dict.py  get_vocab.py  mfcc	RESULTS  utils
conf		  data_test  decode.sh	exp	 local	       path.sh	steps


# Zip and download Model

In [46]:
!zip -r /content/vosk-model-small-en-us-0.15-compile-custom.zip /content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab

  adding: content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab/ (stored 0%)
  adding: content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab/exp/ (stored 0%)
  adding: content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab/exp/make_mfcc/ (stored 0%)
  adding: content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab/exp/make_mfcc/test/ (stored 0%)
  adding: content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab/exp/make_mfcc/test/make_mfcc_test_small.10.log (deflated 74%)
  adding: content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab/exp/make_mfcc/test/make_mfcc_test_small.7.log (deflated 74%)
  adding: content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab/exp/make_mfcc/test/make_mfcc_test_small.5.log (deflated 75%)
  adding: content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab/exp/make_mfcc/test/make_mfcc_test_small.4.log (deflated 74%)
  adding: content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab/exp/make_

Overall zip size

In [47]:
!du -sh /content/vosk-model-small-en-us-0.15-compile-custom.zip

514M	/content/vosk-model-small-en-us-0.15-compile-custom.zip


In [48]:
from google.colab import files
files.download("/content/vosk-model-small-en-us-0.15-compile-custom.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Others

In [25]:
!steps/online/nnet2/extract_ivectors_online.sh --nj 4 \
          data_test/test_small exp/extractor \
          exp/ivectors_test


steps/online/nnet2/extract_ivectors_online.sh --nj 4 data_test/test_small exp/extractor exp/ivectors_test
steps/online/nnet2/extract_ivectors_online.sh: extracting iVectors
run.pl: 4 / 4 failed, log is in exp/ivectors_test/log/extract_ivectors.*.log


In [26]:
!ls /opt/kaldi/egs
!cat /opt/kaldi/egs/README.txt

ls: cannot access '/opt/kaldi/egs': No such file or directory
cat: /opt/kaldi/egs/README.txt: No such file or directory


In [ ]:
# %cd /opt/kaldi/egs/ac
# !wget -c "https://alphacephei.com/vosk/models/vosk-model-en-us-0.22-compile.zip" -O vosk-model-en-us-0.22-compile.zip
# !rm -rf vosk-model-en-us-0.22-compile
# !unzip vosk-model-en-us-0.22-compile

[Errno 2] No such file or directory: '/opt/kaldi/egs/ac'
/content
--2025-03-13 02:03:55--  https://alphacephei.com/vosk/models/vosk-model-en-us-0.22-compile.zip
Resolving alphacephei.com (alphacephei.com)... 188.40.21.16, 2a01:4f8:13a:279f::2
Connecting to alphacephei.com (alphacephei.com)|188.40.21.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2985202126 (2.8G) [application/zip]
Saving to: ‘vosk-model-en-us-0.22-compile.zip’

sk-model-en-us-0.22   8%[>                   ] 243.54M   101MB/s               ^C
Archive:  vosk-model-en-us-0.22-compile.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of vosk-model-en-us-0.22-compile or
        vosk-model-en-us-0.22-compile.zip, and cannot find vosk-model-en-us-

In [ ]:
%cd /content/kaldi/tools/
# !bash extras/install_opengrm.sh.
# !bash extras/install_SRILM.sh

/content/kaldi/tools


In [ ]:
import kora.install.kaldi
# %cd /content/kaldi/src/
# !./configure --shared
# !make depend -j 8
# !make -j 8

In [ ]:
%cd /content/kaldi/egs/ac/vosk-model-en-us-0.22-compile
# !ls
# !cat compile-graph.sh
# print("*" * 50)
!bash compile-graph.sh

/content/kaldi/egs/ac/vosk-model-en-us-0.22-compile
+ rm -rf 'data/*.lm.gz' data/lang_local data/dict data/lang data/lang_test data/lang_test_rescore
+ rm -rf exp/lgraph
+ rm -rf exp/graph
+ mkdir -p data/dict
+ cp db/phone/extra_questions.txt db/phone/nonsilence_phones.txt db/phone/optional_silence.txt db/phone/silence_phones.txt data/dict
+ python3 ./dict.py
+ ngram-count -wbdiscount -order 4 -text db/extra.txt -lm data/extra.lm.gz
compile-graph.sh: line 15: ngram-count: command not found
+ ngram -order 4 -lm db/en-230k-0.5.lm.gz -mix-lm data/extra.lm.gz -lambda 0.95 -write-lm data/en-mix.lm.gz
compile-graph.sh: line 16: ngram: command not found
+ ngram -order 4 -lm data/en-mix.lm.gz -prune 3e-8 -write-lm data/en-mixp.lm.gz
compile-graph.sh: line 17: ngram: command not found
+ ngram -lm data/en-mixp.lm.gz -write-lm data/en-mix-small.lm.gz
compile-graph.sh: line 18: ngram: command not found
+ utils/prepare_lang.sh data/dict '[unk]' data/lang_local data/lang
utils/prepare_lang.sh data/

In [ ]:
%cd /content/kaldi/egs/ac/vosk-model-en-us-0.22-compile
!ls
!cat decode.sh
!bash decode.sh

/content/kaldi/egs/ac/vosk-model-en-us-0.22-compile
cmd.sh		  conf	db	   dict.py  local  path.sh  RESULTS	 rnnlm	utils
compile-graph.sh  data	decode.sh  exp	    mfcc   README   RESULTS.txt  steps
#!/bin/bash

. ./cmd.sh
. ./path.sh
. ./utils/parse_options.sh

for task in test_tedlium; do

    steps/make_mfcc.sh --nj 10 data/${task} exp/make_mfcc/test mfcc
    steps/compute_cmvn_stats.sh data/${task} exp/make_mfcc/test mfcc
    utils/fix_data_dir.sh data/${task}

    steps/online/nnet2/extract_ivectors_online.sh --nj 4 --use-vad false \
      data/${task} exp/chain/extractor \
      exp/chain/ivectors_${task}

    steps/nnet3/decode.sh --cmd $decode_cmd --num-threads 1 --nj 10 \
          --online-ivector-dir exp/chain/ivectors_${task} \
          --acwt 1.0 --post-decode-acwt 10.0 \
         exp/chain/tdnn/graph data/${task} exp/chain/tdnn/decode_${task}

    steps/lmrescore_const_arpa.sh data/lang_test data/lang_test_rescore \
        data/${task} exp/chain/tdnn/decode_${task} exp/cha

In [ ]:
!cat exp/ivectors_test/log/extract_ivectors.*.log

# ivector-extract-online2 --config=exp/ivectors_test/conf/ivector_extractor.conf ark:data_test/test_small/split4/1/spk2utt scp:data_test/test_small/split4/1/feats.scp ark:- | copy-feats --compress=true ark:- ark,scp:/content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab/exp/ivectors_test/ivector_online.1.ark,/content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab/exp/ivectors_test/ivector_online.1.scp 
# Started at Thu Mar 13 02:38:27 UTC 2025
#
ivector-extract-online2: error while loading shared libraries: libcudart.so.11.0: cannot open shared object file: No such file or directory
copy-feats --compress=true ark:- ark,scp:/content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab/exp/ivectors_test/ivector_online.1.ark,/content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab/exp/ivectors_test/ivector_online.1.scp 
LOG (copy-feats[5.5.1092~1-341d0]:main():copy-feats.cc:143) Copied 0 feature matrices.
# Accounting: time=0 threads=1
# Ended (code 1) at Thu Mar